Reproduced first place solution from: https://www.kaggle.com/competitions/open-problems-single-cell-perturbations/leaderboard

All credit goes to Jean!

Steps involved here:

1. Preprocessing:
a) ChemBERT --- for features of small molecules’ SMILES (taken from here: https://huggingface.co/DeepChem/ChemBERTa-77M-MTR)
b) Additional augmentation: mean, standard deviation, and (25%, 50%, 75%) percentiles of differential expressions per cell type and small molecule in the training

Combined and validated produces following pre-processed data:

“initial”: ChemBERTa embeddings, 1 hot encoding of cell_type/sm_name pairs, mean, std, percentiles of targets per cell_type and sm_name

“light”: ChemBERTa embeddings, 1 hot encoding of cell_type/sm_name pairs, mean targets per cell_type and sm_name

“heavy”: ChemBERTa embeddings, 1 hot encoding of cell_type/sm_name pairs, mean, 25%, 50%, 75% percentiles of targets per cell_type and sm_name

This data was validated with 5 fold cross validation across folds. Where ''T regulatory cells’’, ''B cells’’, and ''Nk cells’’ were easiest to predict, while ''T cells CD8+’’ and ''Myeloid cells’’ are the hardest to predict

2. Model: Weighted average between 3 "simple" trained models: LSTM, GRU, and 1-d CNN 

3. Adding noise and making it robust: Randomly replacing 30% of the input features’ entries with zeros, and adding the resulting input feature together with the correct target as a new training datapoint. This has proven to improve the predictive performance of models. In this sense, models are robust to the noise as their performance is not hindered but rather improved. The biological motivation here is that we might not need to know the complete chemical structure of a molecule (assuming the dropped input features are from sm_name) to know its impact on a cell:

'>  def augment_data(x_, y_):
>        copy_x = x_.copy()
>        new_x = []
>        new_y = y_.copy()
>        dim = x_.shape[2]
>        k = int(0.3*dim)
>        for i in range(x_.shape[0]):
>           idx = random.sample(range(dim), k=k)
>           copy_x[i,:,idx] = 0
>           new_x.append(copy_x[i])
>       return np.stack(new_x, axis=0), new_y


4. Post processing (all credict goes here: https://www.kaggle.com/code/raki21/4th-place-magic-postprocessing)


5. Reproduced result on the leaderboard:

Things left to do:

1. Abelation study 
2. categories (genes, cell types, compounds) predicted well and predicted poorly
3. Viash PR (with both train and predict component)
4. Write it all up in overleaf

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import random
import numpy as np 
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader
from transformers import AutoModelForMaskedLM, AutoTokenizer
import json

## Seed Everything

In [2]:
def seed_everything():
    seed = 42
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    print('-----Seed Set!-----') 

In [3]:
seed_everything()

-----Seed Set!-----


## Download the pretrained ChemBERTa model

In [4]:
chemberta = AutoModelForMaskedLM.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
chemberta._modules["lm_head"] = nn.Identity()

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Read Competition Datasets

In [5]:
de_train = pd.read_parquet('../input/open-problems-single-cell-perturbations/de_train.parquet')
id_map = pd.read_csv('../input/open-problems-single-cell-perturbations/id_map.csv')
sample_submission = pd.read_csv('../input/open-problems-single-cell-perturbations/sample_submission.csv', index_col='id')

In [6]:
xlist  = ['cell_type','sm_name']
_ylist = ['cell_type','sm_name','sm_lincs_id','SMILES','control']

y = de_train.drop(columns=_ylist)
y.shape

(614, 18211)

## Use Scikit-Learn's One Hot Encoder
This helps encode each pair (cell_type, sm_name) as a multi-dimensional binary vector

In [7]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(de_train[xlist])
one_hot_encode_features = encoder.transform(de_train[xlist])
one_hot_test = encoder.transform(id_map[xlist])

X = pd.DataFrame(one_hot_encode_features.toarray().astype(float))
test = pd.DataFrame(one_hot_test.toarray().astype(float))

## 

## First Data Augmentation
Compute the mean and std differential expression for each cell type and each small molecule name. Take this as additional input features to the model to be built.

In [8]:
de_cell_type = de_train.iloc[:, [0] + list(range(5, de_train.shape[1]))]
de_sm_name = de_train.iloc[:, [1] + list(range(5, de_train.shape[1]))]
mean_cell_type = de_cell_type.groupby('cell_type').mean().reset_index()
mean_sm_name = de_sm_name.groupby('sm_name').mean().reset_index()

std_cell_type = de_cell_type.groupby('cell_type').std().reset_index()
std_sm_name = de_sm_name.groupby('sm_name').std().reset_index()

I also consider the 25%, 50%, and 75% percentiles (see below). Not that I explored different combinations of these features. In particular, when using all additional features (i.e., mean, std, 25%, 50%, and 75% percentiles) I refer to the corresponding models as "heavy".

In [9]:
desc_cell_type = de_cell_type.groupby('cell_type').describe().reset_index()
desc_cell_type.drop(['mean', 'count', 'std', 'min', 'max'], axis=1, level=1, inplace=True)

## Build a Mapping Between Each SM Name and Its SMILES
This helps build Chem BERTa features for molecule SMILES

In [10]:
sm_name2smiles = {smname:smiles for smname, smiles in zip(de_train['sm_name'], de_train['SMILES'])}
test_smiles = list(map(sm_name2smiles.get, id_map['sm_name'].values))

Below, we define a variable to control whether we want to reproduce the leaderboard or train different, potentially better instances of our proposed models

In [11]:
reproduce_leaderboard_score = True # Either reproduce the leaderboard or build new (different) models to predict

Thanks to ALEKSEY TREPETSKY (I upvoted) https://www.kaggle.com/code/alekseytrepetsky/create-chemberta-embed/notebook, I could either build my own ChemBERTa features or use the ones she/he has created and shared publicly.

In [12]:
chemberta.eval()
def build_ChemBERTa_features(smiles_list):
    embeddings = torch.zeros(len(smiles_list), 384)
    embeddings_mean = torch.zeros(len(smiles_list), 384)

    with torch.no_grad():
        for i, smiles in enumerate(tqdm(smiles_list)):
            encoded_input = tokenizer(smiles, return_tensors="pt", padding=False, truncation=True)
            model_output = chemberta(**encoded_input)
            
            embedding = model_output[0][::,0,::]
            embeddings[i] = embedding
            
            embedding = torch.mean(model_output[0], 1)
            embeddings_mean[i] = embedding
            
    return embeddings.numpy(), embeddings_mean.numpy()

In [13]:
if reproduce_leaderboard_score:
    train_chem_feat = np.load('../input/create-chemberta-embed/train_ChemBERTa_v2_77MTR_cls_pad_True.npy')
    test_chem_feat = np.load('../input/create-chemberta-embed/test_ChemBERTa_v2_77MTR_cls_pad_True.npy')
    train_chem_feat_mean = np.load('../input/create-chemberta-embed/train_ChemBERTa_v2_77MTR_mean_pad_True.npy')
    test_chem_feat_mean = np.load('../input/create-chemberta-embed/test_ChemBERTa_v2_77MTR_mean_pad_True.npy')
else:
    train_chem_feat, train_chem_feat_mean = build_ChemBERTa_features(de_train.SMILES)
    test_chem_feat, test_chem_feat_mean = build_ChemBERTa_features(test_smiles)

## Now Define the Function to Combine the Created Features in Different Ways
For each combination, we create and train 3 deep learning architectures (see below)

In [14]:
def combine_features(add_dfs, chem_feats, main_df, add_vecs=None, use_description=False):
    new_vecs = []
    chem_feat_dim = 600 if reproduce_leaderboard_score else 384
    if len(add_dfs) > 0:
        add_len = sum(add_df.shape[1]-1 for add_df in add_dfs)+chem_feat_dim*len(chem_feats)+add_vecs.shape[1] if\
        add_vecs is not None else sum(add_df.shape[1]-1 for add_df in add_dfs)+chem_feat_dim*len(chem_feats)
    else:
        add_len = chem_feat_dim*len(chem_feats)+add_vecs.shape[1] if\
        add_vecs is not None else chem_feat_dim*len(chem_feats)
    if use_description and reproduce_leaderboard_score:
        add_len += (desc_cell_type.shape[1]-1)//3
    for i in range(len(main_df)):
        if add_vecs is not None:
            vec_ = (add_vecs.iloc[i,:].values).copy()
        else:
            vec_ = np.array([])
        for df in add_dfs:
            if 'cell_type' in df.columns:
                values = df[df['cell_type']==main_df.iloc[i]['cell_type']].values.squeeze()[1:].astype(float)
                vec_ = np.concatenate([vec_, values])
            else:
                assert 'sm_name' in df.columns
                values = df[df['sm_name']==main_df.iloc[i]['sm_name']].values.squeeze()[1:].astype(float)
                vec_ = np.concatenate([vec_, values])
        for chem_feat in chem_feats:
            vec_ = np.concatenate([vec_, chem_feat[i]])
        final_vec = np.concatenate([vec_,np.zeros(add_len-vec_.shape[0],)])
        new_vecs.append(final_vec)
    return np.stack(new_vecs, axis=0).astype(float).reshape(len(main_df), 1, add_len)

In [15]:
X_vec = combine_features([mean_cell_type,std_cell_type,mean_sm_name,std_sm_name],\
                [train_chem_feat, train_chem_feat_mean], de_train, X)
test_vec = combine_features([mean_cell_type,std_cell_type,mean_sm_name,std_sm_name],\
                   [test_chem_feat, test_chem_feat_mean], id_map, test)

X_vec_light = combine_features([mean_cell_type,mean_sm_name],\
                [train_chem_feat, train_chem_feat_mean], de_train, X)
test_vec_light = combine_features([mean_cell_type,mean_sm_name],\
                   [test_chem_feat, test_chem_feat_mean], id_map, test)

In [16]:
if not reproduce_leaderboard_score:
    X_vec = np.concatenate([X_vec, np.zeros((X_vec.shape[0], 1, 2))], axis=-1)
    test_vec = np.concatenate([test_vec, np.zeros((test_vec.shape[0], 1, 2))], axis=-1)
    X_vec_light = np.concatenate([X_vec_light, np.zeros((X_vec_light.shape[0], 1, 1))], axis=-1)
    test_vec_light = np.concatenate([test_vec_light, np.zeros((test_vec_light.shape[0], 1, 1))], axis=-1)

In [17]:
X_vec_light.shape

(614, 1, 37774)

In [18]:
X_vec_heavy = combine_features([desc_cell_type,mean_cell_type,mean_sm_name],\
                [train_chem_feat,train_chem_feat_mean], de_train, X, use_description=True)
test_vec_heavy = combine_features([desc_cell_type,mean_cell_type,mean_sm_name],\
                   [test_chem_feat,test_chem_feat], id_map, test, use_description=True)

In [19]:
test_vec_heavy.shape

(255, 1, 110618)

## Evaluation Metric Function

In [20]:
def mrrmse_np(y_pred, y_true):
    return np.sqrt(np.square(y_true - y_pred).mean(axis=1)).mean()

## Additional Loss Functions
I discovered experimentally that by combining different loss functions (others defined in the models), we can achieve a better performance. In particular, I used the binary cross entropy loss to push each predicted value in the target to a value other than 0 (i.e., push the value to a strictly positive or negative value). This is motivated by the fact that several values in the target are close to zero, and I wanted to make sure models do not learn this naively. The rest of the loss functions are suited for regression tasks and used normally to enforce the predicted value to be close to the target.

In [21]:
class LogCoshLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_prime_t, y_t):
        ey_t = (y_t - y_prime_t)/3 # divide by 3 to avoid numerical overflow in cosh
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))

In [22]:
hidden_dims_reproduce_leaderboard = {'conv': {'heavy': 13400, 'light': 4576, 'initial': 8992},
                                    'rnn': {'linear': {'heavy': 99968, 'light': 24192, 'initial': 29568},
                                           'input_shape': {'heavy': [779,142], 'light': [187,202], 'initial': [229,324]}
                                           }}

hidden_dims_new = {'conv': {'heavy': 11144, 'light': 4520, 'initial': 8936},
                                    'rnn': {'linear': {'heavy': 36480, 'light': 13952, 'initial': 19968},
                                           'input_shape': {'heavy': [283,325], 'light': [107,349], 'initial': [154,479]}
                                           }}

dims_dict = hidden_dims_reproduce_leaderboard if reproduce_leaderboard_score else hidden_dims_new

## Modeling

In [23]:
class Conv(nn.Module):
    def __init__(self, scheme):
        super(Conv, self).__init__()
        self.name = 'Conv'
        self.conv_block = nn.Sequential(nn.Conv1d(1, 8, 5, stride=1, padding=0),
                                        nn.Dropout(0.3),
                                        nn.Conv1d(8, 8, 5, stride=1, padding=0),
                                        nn.ReLU(),
                                        nn.Conv1d(8, 16, 5, stride=2, padding=0),
                                        nn.Dropout(0.3),
                                        nn.AvgPool1d(11),
                                        nn.Conv1d(16, 8, 3, stride=3, padding=0),
                                        nn.Flatten())
        self.scheme = scheme
        self.linear = nn.Sequential(
                nn.Linear(dims_dict['conv'][self.scheme], 1024),
                nn.Dropout(0.3),
                nn.ReLU(),
                nn.Linear(1024, 512),
                nn.Dropout(0.3),
                nn.ReLU())
        self.head1 = nn.Linear(512, 18211)
        
        self.loss1 = nn.MSELoss()
        self.loss2 = LogCoshLoss()
        self.loss3 = nn.L1Loss()
        self.loss4 = nn.BCELoss()
        
    def forward(self, x, y=None):
        if y is None:
            out = self.conv_block(x)
            out = self.head1(self.linear(out))
            return out
        else:
            out = self.conv_block(x)
            out = self.head1(self.linear(out))
            loss1 = 0.4*self.loss1(out, y) + 0.3*self.loss2(out, y) + 0.3*self.loss3(out, y)
            yhat = torch.sigmoid(out)
            yy = torch.sigmoid(y)
            loss2 = self.loss4(yhat, yy)
            return 0.8*loss1 + 0.2*loss2
        

class LSTM(nn.Module):
    def __init__(self, scheme):
        super(LSTM, self).__init__()
        self.name = 'LSTM'
        self.scheme = scheme
        self.lstm = nn.LSTM(dims_dict['rnn']['input_shape'][self.scheme][1], 128, num_layers=2, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(dims_dict['rnn']['linear'][self.scheme], 1024),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Dropout(0.3),
            nn.ReLU())
        self.head1 = nn.Linear(512, 18211)
        
        self.loss1 = nn.MSELoss()
        self.loss2 = LogCoshLoss()
        self.loss3 = nn.L1Loss()
        self.loss4 = nn.BCELoss()
        
    def forward(self, x, y=None):
        shape1, shape2 = dims_dict['rnn']['input_shape'][self.scheme]
        x = x.reshape(x.shape[0],shape1,shape2)
        if y is None:
            out, (hn, cn) = self.lstm(x)
            out = out.reshape(out.shape[0],-1)
            out = torch.cat([out, hn.reshape(hn.shape[1], -1)], dim=1)
            out = self.head1(self.linear(out))
            return out
        else:
            out, (hn, cn) = self.lstm(x)
            out = out.reshape(out.shape[0],-1)
            out = torch.cat([out, hn.reshape(hn.shape[1], -1)], dim=1)
            out = self.head1(self.linear(out))
            loss1 = 0.4*self.loss1(out, y) + 0.3*self.loss2(out, y) + 0.3*self.loss3(out, y)
            yhat = torch.sigmoid(out)
            yy = torch.sigmoid(y)
            loss2 = self.loss4(yhat, yy)
            return 0.8*loss1 + 0.2*loss2
        
        
class GRU(nn.Module):
    def __init__(self, scheme):
        super(GRU, self).__init__()
        self.name = 'GRU'
        self.scheme = scheme
        self.gru = nn.GRU(dims_dict['rnn']['input_shape'][self.scheme][1], 128, num_layers=2, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(dims_dict['rnn']['linear'][self.scheme], 1024),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Dropout(0.3),
            nn.ReLU())
        self.head1 = nn.Linear(512, 18211)
        
        self.loss1 = nn.MSELoss()
        self.loss2 = LogCoshLoss()
        self.loss3 = nn.L1Loss()
        self.loss4 = nn.BCELoss()
        
    def forward(self, x, y=None):
        shape1, shape2 = dims_dict['rnn']['input_shape'][self.scheme]
        x = x.reshape(x.shape[0],shape1,shape2)
        if y is None:
            out, hn = self.gru(x)
            out = out.reshape(out.shape[0],-1)
            out = torch.cat([out, hn.reshape(hn.shape[1], -1)], dim=1)
            out = self.head1(self.linear(out))
            return out
        else:
            out, hn = self.gru(x)
            out = out.reshape(out.shape[0],-1)
            out = torch.cat([out, hn.reshape(hn.shape[1], -1)], dim=1)
            out = self.head1(self.linear(out))
            loss1 = 0.4*self.loss1(out, y) + 0.3*self.loss2(out, y) + 0.3*self.loss3(out, y)
            yhat = torch.sigmoid(out)
            yy = torch.sigmoid(y)
            loss2 = self.loss4(yhat, yy)
            return 0.8*loss1 + 0.2*loss2

## Create Dataset Class

In [24]:
class Dataset:
    def __init__(self, data_x, data_y=None):
        super(Dataset, self).__init__()
        self.data_x = data_x
        self.data_y = data_y

    def __len__(self):
        return len(self.data_x)
    
    def __getitem__(self, idx):
        if self.data_y is not None:
            return self.data_x[idx], self.data_y[idx]
        else:
            return self.data_x[idx]

## Define Data Augmentation Function
In the following function, we augment the training data by randomly dropping 30% of our 1-dimensional input feature vectors' entries. Input features are of shape (batch, 1, d)

In [25]:
import random
def augment_data(x_, y_):
    copy_x = x_.copy()
    new_x = []
    new_y = y_.copy()
    dim = x_.shape[2]
    k = int(0.3*dim)
    for i in range(x_.shape[0]):
        idx = random.sample(range(dim), k=k)
        copy_x[i,:,idx] = 0
        new_x.append(copy_x[i])
    return np.stack(new_x, axis=0), new_y

## Define Helper and Main Training Functions
GRU experienced numerical overflow with a learning rate of 0.001, so I used 0.0003 instead

In [26]:
def train_step(dataloader, model, opt, clip_norm):
    model.train()
    train_losses = []
    for x, target in dataloader:
        if torch.cuda.is_available():
            model.cuda()
            x = x.cuda()
            target = target.cuda()
        loss = model(x, target)
        train_losses.append(loss.item())
        opt.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), clip_norm)
        opt.step()
    return np.mean(train_losses)

def validation_step(dataloader, model):
    model.eval()
    val_losses = []
    val_mrrmse = []
    for x, target in dataloader:
        if torch.cuda.is_available():
            model.cuda()
            x = x.cuda()
            target = target.cuda()
        loss = model(x,target)
        pred = model(x).detach().cpu().numpy()
        val_mrrmse.append(mrrmse_np(pred, target.cpu().numpy()))
        val_losses.append(loss.item())
    return np.mean(val_losses), np.mean(val_mrrmse)


def train_function(model, x_train, y_train, x_val, y_val, epochs=20, clip_norm=1.0):
    if model.name in ['GRU'] or not reproduce_leaderboard_score:
        print('lr', 0.0003)
        opt = torch.optim.Adam(model.parameters(), lr=0.0003)
    else:
        opt = torch.optim.Adam(model.parameters(), lr=0.001)
    model.cuda()
    x_train_aug, y_train_aug = augment_data(x_train, y_train)
    x_train_aug = np.concatenate([x_train, x_train_aug], axis=0)
    y_train_aug = np.concatenate([y_train, y_train_aug], axis=0)
    data_x_train = torch.FloatTensor(x_train_aug)
    data_y_train = torch.FloatTensor(y_train_aug)
    data_x_val = torch.FloatTensor(x_val)
    data_y_val = torch.FloatTensor(y_val)
    train_dataloader = DataLoader(Dataset(data_x_train, data_y_train), num_workers=4, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(Dataset(data_x_val, data_y_val), num_workers=4, batch_size=32, shuffle=False)
    best_loss = np.inf
    best_weights = None
    train_losses = []
    val_losses = []
    for e in range(epochs):
        loss = train_step(train_dataloader, model, opt, clip_norm)
        val_losses.append(loss.item())
        val_loss, val_mrrmse = validation_step(val_dataloader, model)
        if val_mrrmse < best_loss:
            best_loss = val_mrrmse
            best_weights = model.state_dict()
            print('BEST ----> ')
        print(f"{model.name} Epoch {e}, train_loss {round(loss,3)}, val_loss {round(val_loss, 3)}, val_mrrmse {val_mrrmse}")
    model.load_state_dict(best_weights)
    return model

In [27]:
from sklearn.model_selection import KFold as KF
splits = 5
kf_cv = KF(n_splits=splits, shuffle=True, random_state=42)

In [28]:
def cross_validate_models(X, y, epochs=120, scheme='initial', clip_norm=1.0):
    trained_models = []
    for i,(train_idx,val_idx) in enumerate(kf_cv.split(X)):
        print(f"\nSplit {i+1}/{splits}...")
        x_train, x_val = X[train_idx], X[val_idx]
        y_train, y_val = y.values[train_idx], y.values[val_idx]
        for Model in [LSTM, Conv, GRU]:
            model = Model(scheme)
            model = train_function(model, x_train, y_train, x_val, y_val, epochs=epochs, clip_norm=clip_norm)
            model.to('cpu')
            trained_models.append(model)
            torch.cuda.empty_cache()
    return trained_models

## Define Inference Functions

In [29]:
def inference_pytorch(model, dataloader):
    model.eval()
    preds = []
    for x in dataloader:
        if torch.cuda.is_available():
            model.cuda()
            x = x.cuda()
        pred = model(x).detach().cpu().numpy()
        preds.append(pred)
    model.to('cpu')
    torch.cuda.empty_cache()
    return np.concatenate(preds, axis=0)

In [30]:
def average_prediction(X_test, trained_models):
    all_preds = []
    test_dataloader = DataLoader(Dataset(torch.FloatTensor(X_test)), num_workers=4, batch_size=64, shuffle=False)
    for i,model in enumerate(trained_models):
        current_pred = inference_pytorch(model, test_dataloader)
        all_preds.append(current_pred)
    return np.stack(all_preds, axis=1).mean(axis=1)

In [31]:
def weighted_average_prediction(X_test, trained_models, model_wise=[0.25, 0.35, 0.40], fold_wise=None):
    all_preds = []
    test_dataloader = DataLoader(Dataset(torch.FloatTensor(X_test)), num_workers=4, batch_size=64, shuffle=False)
    for i,model in enumerate(trained_models):
        current_pred = inference_pytorch(model, test_dataloader)
        current_pred = model_wise[i%3]*current_pred
        if fold_wise:
            current_pred = fold_wise[i//3]*current_pred
        all_preds.append(current_pred)
    return np.stack(all_preds, axis=1).sum(axis=1)

## Train and Reproduce the Leaderboard!

In [32]:
def reproduce(epochs=1):
    trained_models = {'initial': [], 'light': [], 'heavy': []}
    for scheme, clip_norm, input_features in zip(['initial','light','heavy'], [5.0, 1.0, 1.0], [X_vec, X_vec_light, X_vec_heavy]):
        seed_everything()
        models = cross_validate_models(input_features, y, epochs=epochs, scheme=scheme, clip_norm=clip_norm)
        trained_models[scheme].extend(models)
    return trained_models

In [33]:
trained_models = reproduce(epochs=250)

-----Seed Set!-----

Split 1/5...
BEST ----> 
LSTM Epoch 0, train_loss 2.096, val_loss 1.16, val_mrrmse 0.991895318031311
LSTM Epoch 1, train_loss 1.872, val_loss 2.23, val_mrrmse 1.210561990737915
LSTM Epoch 2, train_loss 2.011, val_loss 1.283, val_mrrmse 1.0085457563400269
LSTM Epoch 3, train_loss 1.703, val_loss 1.212, val_mrrmse 0.9939535856246948
LSTM Epoch 4, train_loss 1.68, val_loss 1.234, val_mrrmse 1.0307337045669556
LSTM Epoch 5, train_loss 1.671, val_loss 1.444, val_mrrmse 1.0314505100250244
LSTM Epoch 6, train_loss 1.577, val_loss 1.361, val_mrrmse 1.0260246992111206
LSTM Epoch 7, train_loss 1.44, val_loss 1.774, val_mrrmse 1.0883088111877441
LSTM Epoch 8, train_loss 1.448, val_loss 1.181, val_mrrmse 0.9931333065032959
LSTM Epoch 9, train_loss 1.412, val_loss 1.966, val_mrrmse 1.090728998184204
LSTM Epoch 10, train_loss 1.378, val_loss 1.568, val_mrrmse 1.0396913290023804
LSTM Epoch 11, train_loss 1.301, val_loss 1.515, val_mrrmse 1.0244438648223877
LSTM Epoch 12, train_lo

In [34]:
pred1 = average_prediction(test_vec, trained_models['initial'])
pred2 = weighted_average_prediction(test_vec, trained_models['initial'],\
                                    model_wise=[0.29, 0.33, 0.38], fold_wise=[0.25, 0.15, 0.2, 0.15, 0.25])

In [35]:
pred3 = average_prediction(test_vec_light, trained_models['light'])
pred4 = weighted_average_prediction(test_vec_light, trained_models['light'],\
                                    model_wise=[0.29, 0.33, 0.38], fold_wise=[0.25, 0.15, 0.2, 0.15, 0.25])

In [36]:
pred5 = average_prediction(test_vec_heavy, trained_models['heavy'])
pred6 = weighted_average_prediction(test_vec_heavy, trained_models['heavy'],\
                                    model_wise=[0.29, 0.33, 0.38], fold_wise=[0.25, 0.15, 0.2, 0.15, 0.25])

## Read Submission Sample File

In [37]:
col = list(de_train.columns[5:])
submission = sample_submission.copy()

In [38]:
submission[col] = 0.18*pred1 + 0.15*pred2 + 0.23*pred3 + 0.15*pred4 + 0.15*pred5 + 0.14*pred6
df1 = submission.copy()

## Ensemble Prediction

In [39]:
submission[col] =  0.23*pred1 + 0.15*pred2 + 0.13*pred3 + 0.15*pred4 + 0.20*pred5 + 0.14*pred6
df2 = submission.copy()

In [40]:
submission[col] = 0.17*pred1 + 0.16*pred2 + 0.17*pred3 + 0.16*pred4 + 0.18*pred5 + 0.16*pred6
df3 = submission.copy()

In [41]:
df_sub = 0.34*df1 + 0.33*df2 + 0.33*df3

## Save Submission Dataframe

In [42]:
df_sub.to_csv('submission.csv')

In [43]:
df_sub

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.431220,0.204304,0.235907,0.553206,1.276552,0.908929,-1.207080e-02,0.289966,-0.085586,0.379022,...,-0.211065,0.047183,-0.042464,0.123361,0.477829,0.362787,0.264231,0.282896,-0.276190,0.040589
1,0.126678,0.084022,0.110098,0.191143,0.411099,0.281977,-2.764451e-02,0.130964,-0.040344,0.125686,...,-0.066286,0.013244,-0.074683,0.087673,0.174893,0.136845,0.147960,0.100047,-0.096026,-0.066539
2,0.453972,0.205501,0.245085,0.578844,1.445402,0.941471,-1.485819e-02,0.240209,0.000047,0.306720,...,-0.109657,0.081212,-0.088158,0.234756,0.453756,0.293844,0.244291,0.199781,-0.174123,-0.031649
3,0.089317,0.069120,0.085070,0.148146,0.304212,0.213113,-3.698963e-02,0.117031,-0.057705,0.108603,...,-0.068455,0.000701,-0.066211,0.064782,0.148242,0.119782,0.136939,0.091471,-0.096721,-0.066936
4,0.131595,0.079445,0.119654,0.214682,0.406764,0.304155,-4.093058e-02,0.149494,-0.065292,0.167606,...,-0.095546,-0.006776,-0.036900,0.067380,0.172304,0.152857,0.173854,0.128512,-0.151712,-0.048244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.061724,0.021368,-0.059040,0.027223,0.207163,0.109053,-1.895083e-02,0.040701,-0.075486,0.057375,...,-0.067656,-0.038587,-0.128493,-0.061276,0.112277,0.068606,0.057628,0.038314,-0.090064,-0.082396
251,0.201037,0.056008,-0.120832,0.230680,1.060937,0.549361,4.504498e-02,0.088583,-0.071042,0.095157,...,-0.243144,-0.136677,-0.158875,-0.253272,0.342076,0.079660,0.025801,0.055652,-0.278458,-0.005996
252,0.100357,0.036045,-0.068380,0.081245,0.430884,0.234329,-5.991821e-07,0.065840,-0.074628,0.067284,...,-0.112692,-0.051849,-0.124976,-0.101464,0.180059,0.066586,0.045893,0.044741,-0.143338,-0.065524


In [44]:
for index, compound_gene_pred in submission.iterrows():
    if index % 25 == 0:
        print(index)
    abs_compound_mean = abs(compound_gene_pred).mean()
    
    compound_gene_pred *= min(abs_compound_mean**0.6, 1)
    submission.loc[index] = compound_gene_pred  
    
    abs_compound_mean = abs(compound_gene_pred).mean()

0
25
50
75
100
125
150
175
200
225
250


In [46]:
submission *= 1.2